# Test if GPU works correctly

In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


2022-07-13 02:22:44.353326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:44.358459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:44.359624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:44.361191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Loading data

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import h5py

import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Input, Lambda, LeakyReLU, Add, Dense, Activation, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform, constant, TruncatedNormal

%matplotlib inline

In [4]:
with h5py.File('processed_data/np_gan_standard.h5', 'r') as hf:
    data_lr = hf['np_lr'][:]
    data_lr_mean = hf['np_lr_mean'][:]
    data_lr_stddev = hf['np_lr_stddev'][:]
    data_hr = hf['np_hr'][:]
    data_hr_mean = hf['np_hr_mean'][:]
    data_hr_stddev = hf['np_hr_stddev'][:]
    
print(data_lr)
print('\n\n')
print(data_hr)  

[[[[-9.77540612e-01  4.69394028e-01]
   [-9.88514364e-01  4.83604759e-01]
   [-9.88690615e-01  4.81826991e-01]
   ...
   [-8.59407544e-01 -1.84500366e-02]
   [-1.00063288e+00 -3.86129171e-02]
   [-1.01613510e+00 -2.85768192e-02]]

  [[-9.58535731e-01  4.69260812e-01]
   [-9.51555133e-01  5.05433917e-01]
   [-9.31993306e-01  4.93905038e-01]
   ...
   [-9.05861676e-01 -2.17433125e-02]
   [-1.00226498e+00 -1.75988879e-02]
   [-1.02119088e+00 -1.55116441e-02]]

  [[-8.74235630e-01  4.92436051e-01]
   [-8.93230438e-01  5.30668914e-01]
   [-8.97067606e-01  4.91552979e-01]
   ...
   [-8.94055903e-01 -8.23918544e-03]
   [-9.50336456e-01 -9.58461594e-03]
   [-9.82353866e-01 -3.23360483e-03]]

  ...

  [[-5.32296360e-01 -9.07888860e-02]
   [-6.16073012e-01 -1.89724326e-01]
   [-6.43677831e-01 -2.43733719e-01]
   ...
   [-3.77181351e-01  3.39569092e-01]
   [-4.33952421e-01  2.34625027e-01]
   [-4.43929911e-01  1.32707179e-01]]

  [[-7.47961998e-01 -2.87435383e-01]
   [-7.92617738e-01 -3.74220848e

In [5]:
print(data_lr.shape)
print(data_lr_mean)
print(data_lr_stddev)
print('\n\n')
print(data_hr.shape)
print(data_hr_mean)
print(data_hr_stddev)  

(8520, 96, 96, 2)
[ 0.7051484 -1.0147774]
[3.1869051 2.8827915]



(8520, 192, 192, 2)
[ 0.701198  -1.0068085]
[3.149407  2.8781955]


In [6]:
#First split data into train+validation and test set
x_train, x_test, y_train, y_test = train_test_split(data_lr, data_hr, test_size=0.2, random_state=42)

#Next split training again into train and validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

print(np.max(x_train), np.max(x_val), np.max(x_test), np.min(x_train), np.min(x_val), np.min(x_test))
print(np.max(y_train), np.max(y_val), np.max(y_test), np.min(y_train), np.min(y_val), np.min(y_test))

(5112, 96, 96, 2)
(1704, 96, 96, 2)
(1704, 96, 96, 2)
(5112, 192, 192, 2)
(1704, 192, 192, 2)
(1704, 192, 192, 2)
6.890482 5.989441 5.7024393 -5.4777956 -5.0524907 -5.5057893
7.035497 5.8600492 6.0442076 -5.315754 -5.2317853 -5.2996855


In [7]:
def generator(input_shape = (96, 96, 2), nf = 64, r = 2):
    """
    Arguments:
    input_shape -- shape of the images of the dataset, H*W*C
    nf -- integer, the number of filters in all convT layer before super-resolution step
    r -- integer, resolution ratio between output and input

    Returns:
    model -- a Model() instance in Keras
    """
    
    C0 = input_shape[2]
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Define kernel size and stride used
    k, stride = 3, 1
    
    # Shall we use a mirror padding and finally cutoff the edge, like the paper does? FIXME
    X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X_input)
    # Shall we use relu, or leaky_relu? FIXME
    X = Activation('relu')(X)

    skip_connection = X
    
    for i in range(16):
        X_shortcut = X
        
        X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
        X = Activation('relu')(X)
        X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
        X = Add()([X, X_shortcut])
        # Are we missing a relu activation here, if we follow the resnet paper? FIXME
    
    X = Conv2DTranspose(filters=nf, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    X = Add()([X, skip_connection])
    
    # Start to perform sr
    nf_sr = (r**2) * nf
    X = Conv2DTranspose(filters=nf_sr, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    
    sub_layer = Lambda(lambda x:tf.nn.depth_to_space(x,r))
    X = sub_layer(X)
    X = Activation('relu')(X)
    
    X = Conv2DTranspose(filters=C0, kernel_size=(k, k), strides=(stride, stride), padding='same')(X)
    
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [8]:
generator_model = generator(input_shape = (96, 96, 2))

print(generator_model.summary())

2022-07-13 02:22:46.895639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:46.897432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:46.898997: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:46.900121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-13 02:22:46.901222: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 96, 96, 64)  1216        ['input_1[0][0]']                
 ose)                                                                                             
                                                                                                  
 activation (Activation)        (None, 96, 96, 64)   0           ['conv2d_transpose[0][0]']       
                                                                                                  
 conv2d_transpose_1 (Conv2DTran  (None, 96, 96, 64)  36928       ['activation[0][0]']         

                                                                                                  
 add_6 (Add)                    (None, 96, 96, 64)   0           ['conv2d_transpose_14[0][0]',    
                                                                  'add_5[0][0]']                  
                                                                                                  
 conv2d_transpose_15 (Conv2DTra  (None, 96, 96, 64)  36928       ['add_6[0][0]']                  
 nspose)                                                                                          
                                                                                                  
 activation_8 (Activation)      (None, 96, 96, 64)   0           ['conv2d_transpose_15[0][0]']    
                                                                                                  
 conv2d_transpose_16 (Conv2DTra  (None, 96, 96, 64)  36928       ['activation_8[0][0]']           
 nspose)  

                                                                                                  
 activation_15 (Activation)     (None, 96, 96, 64)   0           ['conv2d_transpose_29[0][0]']    
                                                                                                  
 conv2d_transpose_30 (Conv2DTra  (None, 96, 96, 64)  36928       ['activation_15[0][0]']          
 nspose)                                                                                          
                                                                                                  
 add_14 (Add)                   (None, 96, 96, 64)   0           ['conv2d_transpose_30[0][0]',    
                                                                  'add_13[0][0]']                 
                                                                                                  
 conv2d_transpose_31 (Conv2DTra  (None, 96, 96, 64)  36928       ['add_14[0][0]']                 
 nspose)  

In [9]:
#First do pretrain generator for better convergence

adam = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
generator_model.compile(optimizer=adam, loss=losses.MeanSquaredError())

logdir = "gan_v1/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'gan_v1/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [10]:
training_history = generator_model.fit(x_train, y_train,
                                       batch_size=128,
                                       epochs=1000,
                                       shuffle=True,
                                       validation_data=(x_val, y_val),
                                       callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/1000


2022-07-13 02:22:50.879818: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


40/40 [==============================] - 35s 757ms/step - loss: 0.4259 - val_loss: 0.0936
Epoch 2/1000
40/40 [==============================] - 28s 707ms/step - loss: 0.0737 - val_loss: 0.0598
Epoch 3/1000
40/40 [==============================] - 29s 715ms/step - loss: 0.0526 - val_loss: 0.0470
Epoch 4/1000
40/40 [==============================] - 29s 716ms/step - loss: 0.0432 - val_loss: 0.0401
Epoch 5/1000
40/40 [==============================] - 29s 716ms/step - loss: 0.0376 - val_loss: 0.0358
Epoch 6/1000
40/40 [==============================] - 28s 713ms/step - loss: 0.0339 - val_loss: 0.0326
Epoch 7/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0314 - val_loss: 0.0307
Epoch 8/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0295 - val_loss: 0.0290
Epoch 9/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0281 - val_loss: 0.0281
Epoch 10/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0271 -

Epoch 80/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0170 - val_loss: 0.0172
Epoch 81/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0172 - val_loss: 0.0172
Epoch 82/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0169 - val_loss: 0.0171
Epoch 83/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0171 - val_loss: 0.0172
Epoch 84/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0168 - val_loss: 0.0170
Epoch 85/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0169 - val_loss: 0.0171
Epoch 86/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0170 - val_loss: 0.0177
Epoch 87/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0172 - val_loss: 0.0177
Epoch 88/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0169 - val_loss: 0.0169
Epoch 89/1000
40/40 [==============================] - 28s 710ms

40/40 [==============================] - 28s 709ms/step - loss: 0.0143 - val_loss: 0.0145
Epoch 237/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0144 - val_loss: 0.0146
Epoch 238/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0144 - val_loss: 0.0148
Epoch 239/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0149 - val_loss: 0.0152
Epoch 240/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0143 - val_loss: 0.0147
Epoch 241/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0142 - val_loss: 0.0145
Epoch 242/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0141 - val_loss: 0.0146
Epoch 243/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0141 - val_loss: 0.0147
Epoch 244/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0142 - val_loss: 0.0146
Epoch 245/1000
40/40 [==============================] - 28s 710ms/step

40/40 [==============================] - 28s 709ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 393/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0127 - val_loss: 0.0133
Epoch 394/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0127 - val_loss: 0.0131
Epoch 395/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0127 - val_loss: 0.0133
Epoch 396/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 397/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 398/1000
40/40 [==============================] - 28s 709ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 399/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0126 - val_loss: 0.0133
Epoch 400/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0125 - val_loss: 0.0131
Epoch 401/1000
40/40 [==============================] - 28s 709ms/step

40/40 [==============================] - 28s 709ms/step - loss: 0.0118 - val_loss: 0.0129
Epoch 549/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0117 - val_loss: 0.0124
Epoch 550/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0115 - val_loss: 0.0126
Epoch 551/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0115 - val_loss: 0.0124
Epoch 552/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0114 - val_loss: 0.0123
Epoch 553/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0114 - val_loss: 0.0124
Epoch 554/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0114 - val_loss: 0.0123
Epoch 555/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0115 - val_loss: 0.0124
Epoch 556/1000
40/40 [==============================] - 28s 710ms/step - loss: 0.0115 - val_loss: 0.0126
Epoch 557/1000
40/40 [==============================] - 28s 710ms/step

40/40 [==============================] - 28s 712ms/step - loss: 0.0105 - val_loss: 0.0121
Epoch 705/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0106 - val_loss: 0.0122
Epoch 706/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0106 - val_loss: 0.0120
Epoch 707/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0106 - val_loss: 0.0120
Epoch 708/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0106 - val_loss: 0.0121
Epoch 709/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0106 - val_loss: 0.0120
Epoch 710/1000
40/40 [==============================] - 28s 713ms/step - loss: 0.0106 - val_loss: 0.0120
Epoch 711/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0105 - val_loss: 0.0121
Epoch 712/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0106 - val_loss: 0.0121
Epoch 713/1000
40/40 [==============================] - 28s 711ms/step

40/40 [==============================] - 28s 713ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 861/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0119
Epoch 862/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 863/1000
40/40 [==============================] - 28s 711ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 864/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 865/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0119
Epoch 866/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0119
Epoch 867/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0120
Epoch 868/1000
40/40 [==============================] - 28s 712ms/step - loss: 0.0098 - val_loss: 0.0121
Epoch 869/1000
40/40 [==============================] - 28s 712ms/step

In [81]:
generator_model.evaluate(x_test, y_test)

54/54 [==============================] - 3s 52ms/step - loss: 0.0120


0.012037176638841629

# Operations after that should no longer be adopted!!!

In [84]:
#Next do pretrain discriminator for better convergence

adam = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
disc_model.compile(optimizer=adam, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))

logdir = "gan_v2/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_filepath = 'gan_v2/ckp/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                               save_weights_only=True,
                                                               save_freq=10*40)

In [85]:
sr_train_temp = generator_model.predict(x_train)
print(sr_train_temp.shape)

sr_val_temp = generator_model.predict(x_val)
print(sr_val_temp.shape)

160/160 [==============================] - 8s 51ms/step
(5112, 192, 192, 2)
54/54 [==============================] - 3s 51ms/step
(1704, 192, 192, 2)


In [86]:
y_true_label_train = np.ones(y_train.shape[0])
print(y_true_label_train.shape, y_true_label_train)

y_false_label_train = np.zeros(sr_train_temp.shape[0])
print(y_false_label_train.shape, y_false_label_train)

y_true_label_val = np.ones(y_val.shape[0])
print(y_true_label_val.shape, y_true_label_val)

y_false_label_val = np.zeros(sr_val_temp.shape[0])
print(y_false_label_val.shape, y_false_label_val)

(5112,) [1. 1. 1. ... 1. 1. 1.]
(5112,) [0. 0. 0. ... 0. 0. 0.]
(1704,) [1. 1. 1. ... 1. 1. 1.]
(1704,) [0. 0. 0. ... 0. 0. 0.]


In [87]:
disc_y_train = np.concatenate((y_train, sr_train_temp), axis=0)
print(disc_y_train.shape)

disc_y_label_train = np.concatenate((y_true_label_train, y_false_label_train), axis=0)
print(disc_y_label_train.shape)

disc_y_val = np.concatenate((y_val, sr_val_temp), axis=0)
print(disc_y_val.shape)

disc_y_label_val = np.concatenate((y_true_label_val, y_false_label_val), axis=0)
print(disc_y_label_val.shape)

(10224, 192, 192, 2)
(10224,)
(3408, 192, 192, 2)
(3408,)


In [88]:
training_history_disc = disc_model.fit(disc_y_train, disc_y_label_train,
                                       batch_size=128,
                                       epochs=1000,
                                       shuffle=True,
                                       validation_data=(disc_y_val, disc_y_label_val),
                                       callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/1000
80/80 [==============================] - 10s 114ms/step - loss: 0.6903 - val_loss: 0.6861
Epoch 2/1000
80/80 [==============================] - 7s 94ms/step - loss: 0.5332 - val_loss: 0.2979
Epoch 3/1000
80/80 [==============================] - 8s 94ms/step - loss: 0.1340 - val_loss: 0.0639
Epoch 4/1000
80/80 [==============================] - 8s 94ms/step - loss: 0.0300 - val_loss: 0.0172
Epoch 5/1000
80/80 [==============================] - 8s 99ms/step - loss: 0.0161 - val_loss: 0.0115
Epoch 6/1000
80/80 [==============================] - 8s 94ms/step - loss: 0.0063 - val_loss: 0.0061
Epoch 7/1000
80/80 [==============================] - 8s 95ms/step - loss: 0.0049 - val_loss: 0.0045
Epoch 8/1000
80/80 [==============================] - 8s 95ms/step - loss: 0.0022 - val_loss: 0.0061
Epoch 9/1000
80/80 [==============================] - 8s 95ms/step - loss: 0.0996 - val_loss: 0.0115
Epoch 10/1000
80/80 [==============================] - 8s 103ms/step - loss: 0.0313 - val

80/80 [==============================] - 8s 94ms/step - loss: 6.1789e-05 - val_loss: 9.6167e-04
Epoch 154/1000
80/80 [==============================] - 7s 94ms/step - loss: 5.0259e-05 - val_loss: 0.0012
Epoch 155/1000
80/80 [==============================] - 8s 101ms/step - loss: 4.6608e-05 - val_loss: 9.9245e-04
Epoch 156/1000
80/80 [==============================] - 8s 94ms/step - loss: 8.1611e-05 - val_loss: 0.0010
Epoch 157/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.1819e-05 - val_loss: 9.2923e-04
Epoch 158/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.8634e-05 - val_loss: 9.0677e-04
Epoch 159/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.8864e-05 - val_loss: 9.4714e-04
Epoch 160/1000
80/80 [==============================] - 8s 100ms/step - loss: 3.4281e-05 - val_loss: 0.0010
Epoch 161/1000
80/80 [==============================] - 7s 94ms/step - loss: 3.9509e-05 - val_loss: 9.0674e-04
Epoch 162/1000
80/80 [====

80/80 [==============================] - 8s 94ms/step - loss: 4.7498e-06 - val_loss: 0.0014
Epoch 230/1000
80/80 [==============================] - 8s 101ms/step - loss: 5.6386e-06 - val_loss: 0.0011
Epoch 231/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.5538e-06 - val_loss: 8.8109e-04
Epoch 232/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.0110e-06 - val_loss: 9.4529e-04
Epoch 233/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.7596e-06 - val_loss: 9.5718e-04
Epoch 234/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.6571e-06 - val_loss: 8.9089e-04
Epoch 235/1000
80/80 [==============================] - 8s 100ms/step - loss: 3.5127e-06 - val_loss: 8.6937e-04
Epoch 236/1000
80/80 [==============================] - 7s 93ms/step - loss: 3.3000e-06 - val_loss: 7.8627e-04
Epoch 237/1000
80/80 [==============================] - 7s 94ms/step - loss: 3.1408e-06 - val_loss: 8.2114e-04
Epoch 238/1000
80/80 [

Epoch 304/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.6684e-04 - val_loss: 0.0020
Epoch 305/1000
80/80 [==============================] - 8s 100ms/step - loss: 1.5933e-04 - val_loss: 0.0021
Epoch 306/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.3954e-04 - val_loss: 0.0020
Epoch 307/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.3274e-04 - val_loss: 0.0020
Epoch 308/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.3559e-04 - val_loss: 0.0019
Epoch 309/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.6708e-04 - val_loss: 0.0024
Epoch 310/1000
80/80 [==============================] - 8s 101ms/step - loss: 0.1162 - val_loss: 0.0194
Epoch 311/1000
80/80 [==============================] - 8s 95ms/step - loss: 0.0084 - val_loss: 0.0078
Epoch 312/1000
80/80 [==============================] - 8s 94ms/step - loss: 0.0034 - val_loss: 0.0052
Epoch 313/1000
80/80 [=========================

80/80 [==============================] - 8s 101ms/step - loss: 1.2393e-07 - val_loss: 9.2800e-04
Epoch 456/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.1812e-07 - val_loss: 9.3652e-04
Epoch 457/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.1820e-07 - val_loss: 9.0199e-04
Epoch 458/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.1202e-07 - val_loss: 8.9497e-04
Epoch 459/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.0323e-07 - val_loss: 8.3276e-04
Epoch 460/1000
80/80 [==============================] - 8s 100ms/step - loss: 1.0012e-07 - val_loss: 8.5021e-04
Epoch 461/1000
80/80 [==============================] - 8s 94ms/step - loss: 1.0002e-07 - val_loss: 8.8457e-04
Epoch 462/1000
80/80 [==============================] - 8s 94ms/step - loss: 8.8846e-08 - val_loss: 0.0010
Epoch 463/1000
80/80 [==============================] - 8s 94ms/step - loss: 9.0782e-08 - val_loss: 9.2517e-04
Epoch 464/1000
80/

80/80 [==============================] - 8s 94ms/step - loss: 4.9872e-09 - val_loss: 9.8023e-04
Epoch 530/1000
80/80 [==============================] - 8s 101ms/step - loss: 4.5929e-09 - val_loss: 9.0481e-04
Epoch 531/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.3558e-09 - val_loss: 8.7717e-04
Epoch 532/1000
80/80 [==============================] - 7s 94ms/step - loss: 4.4080e-09 - val_loss: 8.9084e-04
Epoch 533/1000
80/80 [==============================] - 7s 94ms/step - loss: 4.0379e-09 - val_loss: 8.6475e-04
Epoch 534/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.8291e-09 - val_loss: 9.4557e-04
Epoch 535/1000
80/80 [==============================] - 8s 100ms/step - loss: 3.6229e-09 - val_loss: 0.0010
Epoch 536/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.4046e-09 - val_loss: 8.8276e-04
Epoch 537/1000
80/80 [==============================] - 8s 94ms/step - loss: 3.2604e-09 - val_loss: 8.4797e-04
Epoch 538/1000
80/

Epoch 680/1000
80/80 [==============================] - 8s 101ms/step - loss: 8.2393e-05 - val_loss: 0.0020
Epoch 681/1000
80/80 [==============================] - 8s 94ms/step - loss: 7.5062e-05 - val_loss: 0.0020
Epoch 682/1000
80/80 [==============================] - 7s 94ms/step - loss: 6.4713e-05 - val_loss: 0.0018
Epoch 683/1000
80/80 [==============================] - 8s 94ms/step - loss: 5.8613e-05 - val_loss: 0.0019
Epoch 684/1000
80/80 [==============================] - 8s 94ms/step - loss: 7.1034e-05 - val_loss: 0.0019
Epoch 685/1000
80/80 [==============================] - 8s 100ms/step - loss: 4.9558e-05 - val_loss: 0.0017
Epoch 686/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.6471e-05 - val_loss: 0.0017
Epoch 687/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.2664e-05 - val_loss: 0.0018
Epoch 688/1000
80/80 [==============================] - 8s 94ms/step - loss: 4.0701e-05 - val_loss: 0.0017
Epoch 689/1000
80/80 [=============

KeyboardInterrupt: 